#       Shelter Animal Outcomes 4

## Logistic Regression

In [1]:
from sklearn.linear_model import LogisticRegression
from sklearn import cross_validation
from sklearn.cross_validation import StratifiedKFold
from sklearn.feature_selection import RFECV
from sklearn.grid_search import GridSearchCV
from sklearn.pipeline import Pipeline
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
df_train = pd.read_csv('../Shelter_train.csv')
df_test = pd.read_csv('../Shelter_test.csv')

In [3]:
X = df_train.ix[:, :-1]
y = df_train.ix[:, -1]
df_test = df_test.drop('ID', 1)

In [4]:
clf = LogisticRegression()
cross_validation.cross_val_score(clf, X, y, scoring="log_loss")

array([-1.02051202, -1.01842145, -1.02075702])

In [5]:
pipeline = Pipeline([
        ('featureSelection', RFECV(estimator=LogisticRegression(), scoring='log_loss')),
        ('clf', LogisticRegression())
    ])
pipeline.fit(X, y)
params = {
    'clf__C': [1, 3, 10, 30, 100, 300, 1000]
}
grid_search = GridSearchCV(pipeline, params)
grid_search.fit(X, y)
grid_search.best_score_
predictions = grid_search.predict_proba(df_test)
output = pd.DataFrame(predictions, columns=['Adoption', 'Died', 'Euthanasia', 'Return_to_owner', 'Transfer'])
output.index.names = ['ID']
output.index += 1
output.head()

,Adoption,Died,Euthanasia,Return_to_owner,Transfer
1,0.092410,0.006379,0.054544,0.248936,0.597731
2,0.446546,0.002548,0.055366,0.334956,0.160584
3,0.675184,0.004908,0.030746,0.048389,0.240774
4,0.243555,0.005632,0.062276,0.254022,0.434515
5,0.586068,0.001482,0.019240,0.308967,0.084242


In [6]:
output.to_csv('../submission-logisticRegression.3.0.csv', index_label = 'ID')